# Lydia link watcher

The second tool needed to automate a shotgun from a lydia collect form is to be able to watch an internet page and watch for a collect link popping ! 

-  I think that most events people apply on Lydia pop on facebook, so I will look for a **Facebook** page watcher. 

-  I will use selenium and a webdriver for this one too. 

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys #used to simulate some keyboard keys (Alt, Tab, etc.)
from selenium.webdriver.common.by import By #used to locate elements on website
import time

from selenium.webdriver.chrome.options import Options #We need this to prevent notifications and pop-ups

We'll first need to login in facebook

-> Let's get our credentials

In [6]:
user = os.getenv("fb_user")
pwd = os.getenv("fb_pwd")

Then, let's launch our web browser.

- We first define our options in order to block any pop-up

In [3]:
option = Options()

option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")

# Pass the argument 1 to allow and 2 to block
option.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 2
})

driver = webdriver.Chrome(chrome_options=option, executable_path='/Users/hugocaetano/Desktop/lydia_collect_automator/chromedriver')
driver.get('https://fb.me/e/50Ng5qZUt') 

/var/folders/f9/pyjz1fyn319c79776sblqzp00000gn/T/ipykernel_31924/4134006082.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=option, executable_path='/Users/hugocaetano/Desktop/lydia_collect_automator/chromedriver')
/var/folders/f9/pyjz1fyn319c79776sblqzp00000gn/T/ipykernel_31924/4134006082.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=option, executable_path='/Users/hugocaetano/Desktop/lydia_collect_automator/chromedriver')


We need to accept the cookies...

In [4]:
cook = driver.find_element(By.XPATH, "//button[@data-cookiebanner='accept_only_essential_button']")
cook.click()

In [7]:
mail = driver.find_element(By.ID, "email")
password = driver.find_element(By.ID, "pass")
mail.send_keys(user)
password.send_keys(pwd)

Now we just need to click ! 

In [8]:
clicker = driver.find_element(By.XPATH, "//input[@value='Se connecter']") #actually this id changes each time so I need another solution...
clicker.click() 

> After this step we are totally logged in. Note that we could totally input manually our username / password, but doing everything automatically is a good habit I think

- Well, now we need to know how to locate a post with a collect link. 

- Then, we'll need to know how to refresh a page and watch for a link in the 3 last posts for example. 

## Step 1 : Locate element where collect.io in post

In [19]:
lydia_post = driver.find_element(By.CSS_SELECTOR, "a[href*='collecte.io']") #here we goooooo :)

In [21]:
lydia_post.click()

In [11]:
driver.close()

Ok, well step 1 was quick. Now we need to look for this link...

I'm thinking of a multiple steps function process : 
- The function would receive as input : 
    - A time duration during which the bot would look for the link (it can be 1min for example)
    - A time step to refresh the page (being let's say 3 seconds by default (but the page needs to load...))
    - The facebook page we need to look for
- Then, the function would do : 
    - Try except find element with collect.io.
    - Wait step time
    - Repeat until time duration is over.

The only thing we have to check is if we can look if an element with a collect link exists quickly enough. Actually I tried and it takes no time ! So let's do this final step !

In [28]:
from selenium.common.exceptions import NoSuchElementException #to try and catch the error we need to import it !

The case where it doesn't work

In [29]:
try :
    driver.find_element(By.CSS_SELECTOR, "a[href*='doesntexist']")
except NoSuchElementException : 
    print("The link you provided was not found.")


The link you provided was not found


The case where it works


In [32]:
try : 
    lydia = driver.find_element(By.CSS_SELECTOR, "a[href*='collecte.io']")
except NoSuchElementException :
    print("The link you provided was not found.")

In [59]:
def pagewatcher(waitingtime = 90, step = 5, link = "collecte.io", driver = driver) :
    """This function looks """
    time0 = time.time()
    while time.time() - time0 < waitingtime :
        try : 
            lydia = driver.find_element(By.CSS_SELECTOR, "a[href*='" + link + "']").click()
            print("the link is clicked on")#Here we would call the lydia form filler function but we need to make it work together first
            m = 1
            break #if we find the link, stop looking for it 
        except NoSuchElementException :
            print("The link you provided was not found.")
            m = 2
            driver.refresh() #if we don't refresh, we won't see the new posts
        time.sleep(step)
    if m == 1 :
        print("it seems like we can complete the form")
    elif m == 2 : 
        print("The link was never found")

In [62]:
driver.refresh()

In [63]:
pagewatcher()

The link you provided was not found.
The link you provided was not found.
The link you provided was not found.
The link you provided was not found.
the link is clicked on
it seems like we can complete the form


Ok, this works so well !

In [55]:
driver.close()

In [10]:
lydia = driver.find_element(By.CSS_SELECTOR, "a[href*='" + "collecte.io" + "']")

In [12]:
lydia.get_attribute("href")

'https://l.facebook.com/l.php?u=https%3A%2F%2Fcollecte.io%2Fsimulation-de-sat-2231239%2Ffr%3Ffbclid%3DIwAR242T1u8_GpYkofJxPJ5EZb_l0ffKgSmvKbn1uUJIT-pwHYstYjQOVXne0&h=AT3wQu7zFC_aU6VptTe7lPk7zAJunTFzWlylc75TkEkt4ES6fV8lqjUU-6RXv6RPrHOmogBWciwShVbOhB7AUZbMaUQ7xQt6Ty-XW3dbg1ajtNnINoEGnSIvMOOoO-hSlDpz9Zn7Ew&__tn__=-UK-R&c[0]=AT3tkyyR428EshsA-FsXVsG87o9LRkuR86pRtnQHMb6PRIt4HLMSwbEOUgBkOJIlg89kz_adlUFYBtmmPlIk9rUnk3rPoIb_eJOAKHrzDaIrE-UEsvEKt7VHYVMmu0GBpKNmg9bt0shRkQwoCa2yWoOftWG9'

In [ ]:
lydia_dr = driver.get(lydia.get_attribute("href"))